In [21]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [22]:
tr = pd.read_csv('train.csv')
te = pd.read_csv('test.csv')

### Random Forest

As we are using time series data, our variables are correlated. We cannot use any linear predictors. 

Adv:
1. The random forest algorithm is not biased, since, there are multiple trees and each tree is trained on a subset of data. Basically, the random forest algorithm relies on the power of "the crowd"; therefore the overall biasedness of the algorithm is reduced.
2. This algorithm is very stable. Even if a new data point is introduced in the dataset the overall algorithm is not affected much since new data may impact one tree, but it is very hard for it to impact all the trees.
3. The random forest algorithm works well when you have both categorical and numerical features.
4. The random forest algorithm also works well when data has missing values or it has not been scaled well (although we have performed feature scaling in this article just for the purpose of demonstration).

Disadv:
1. A major disadvantage of random forests lies in their complexity. They required much more computational resources, owing to the large number of decision trees joined together.
2. Due to their complexity, they require much more time to train than other comparable algorithms.

source: https://stackabuse.com/random-forest-algorithm-with-python-and-scikit-learn/#:~:text=%20The%20following%20are%20the%20basic%20steps%20involved,value%20for%20Y%20%28output%29.%20The%20final...%20More%20)

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor 
from math import sqrt

#fourier transform time.. keep low frequency vs high frequency

In [24]:
tr.columns

Index(['Unnamed: 0', 'Timestamp', 'Values', 'Temperature', 'holiday',
       'Weekday', 'Hour', 'Month', 'Day', 'Time Delta', 'value delta',
       'tv delta', 'prev value', 'twice prev value', 'day shift',
       'month shift', 'year'],
      dtype='object')

#### Initial random forest fit 

In [25]:
#selecting X and Y variables
y_train = tr['tv delta']
X_train = tr[['Temperature', 'holiday', 'prev value','twice prev value','day shift','month shift']]

y_test = te['tv delta']
X_test = te[['Temperature', 'holiday', 'prev value','twice prev value','day shift','month shift']]

In [44]:
from sklearn.preprocessing import StandardScaler

#scaling the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# create regressor object 
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0) 
  
# fit the regressor with x and y data 
regressor.fit(X_train, y_train)

tr_pred = regressor.predict(X_train) 
te_pred = regressor.predict(X_test) 

print("RMSE for training set:", sqrt(mean_squared_error(y_train, tr_pred)))
print("RMSE for test set:", sqrt(mean_squared_error(y_test, te_pred)))

print('Mean Absolute Error for train:', metrics.mean_absolute_error(y_train, tr_pred))
print('Mean Absolute Error for test:', metrics.mean_absolute_error(y_test, te_pred))

feature_list = ['Temperature', 'holiday', 'prev value','twice prev value','day shift','month shift']
# Get numerical feature importances
importances = list(regressor.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

RMSE for training set: 5.716350014724802
RMSE for test set: 15.887416299573475
Mean Absolute Error for train: 3.949370992318209
Mean Absolute Error for test: 11.002088145230802
Variable: prev value           Importance: 0.9
Variable: Temperature          Importance: 0.03
Variable: day shift            Importance: 0.03
Variable: twice prev value     Importance: 0.02
Variable: month shift          Importance: 0.02
Variable: holiday              Importance: 0.0


#### We are clearly overfitting. Let's finetune some parameters

- n_estimators: number of trees built before taking avg of predictions. In general, a higher number of trees increases the performance and makes the predictions more stable, but it also slows down the computation

- min_sample_leaf: The minimum number of samples required to be at a leaf node. A smaller leaf makes the model more prone to capturing noise in train data. We are overfitting so let's try increasing this.

source: https://builtin.com/data-science/random-forest-algorithm

In [27]:
def tuning(X_train,y_train,X_test,y_test, estimators, leaf):
    ''' This function takes a list of estimator and leaf parameters and returns 
    the parameters that produced the lowest error'''
    
    arr = np.zeros((len(estimators),len(leaf))) #empty array to store scores
    
    #scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    for i in range(len(estimators)):
        for j in range(len(leaf)):
            est = estimators[i]
            l = leaf[j]
           
             # create regressor object 
            regressor = RandomForestRegressor(n_estimators = est, min_samples_leaf=l, random_state = 0) 
  
            # fit the regressor with x and y data 
            regressor.fit(X_train, y_train)

            tr_pred = regressor.predict(X_train) 
            te_pred = regressor.predict(X_test) 

            test_score = sqrt(mean_squared_error(y_test, te_pred))
            arr[i][j] = test_score
    
   
    max_ = np.amin(arr) 
    inputs = np.where(arr == max_)
    print("Lowest test RMSE score:",max_)
    best_est = estimators[inputs[0][0]]
    best_leaf = leaf[inputs[1][0]]
    print("Best # estimators:",best_est)
    print("Best # min samples leaf:",best_leaf)

In [28]:
est = [10,100,200,300]
leaf = [15,30,50,70]
max_feats = ['auto', 'sqrt', 'log2']

arr = tuning(X_train,y_train,X_test,y_test,est,leaf)


Lowest test RMSE score: 15.140363398825533
Best # estimators: 300
Best # min samples leaf: 50


Our test RMSE dropped from the previous value of 15.88 to 15.14

Let's see if increasing both the number of estimators and min samples leaf value will help

In [29]:
est = [300,500,700]
leaf = [50,60,70]

arr = tuning(X_train,y_train,X_test,y_test,est,leaf)

Lowest test RMSE score: 15.140363398825533
Best # estimators: 300
Best # min samples leaf: 50


300 and 50 is our best bet. Let's see whether 200 will perform as well as 300. 

In [30]:
regressor = RandomForestRegressor(n_estimators = 200, min_samples_leaf=50, random_state = 0) 
  
# fit the regressor with x and y data 
regressor.fit(X_train, y_train)

tr_pred = regressor.predict(X_train) 
te_pred = regressor.predict(X_test) 

print("RMSE for training set:", sqrt(mean_squared_error(y_train, tr_pred)))
print("RMSE for test set:", sqrt(mean_squared_error(y_test, te_pred)))

RMSE for training set: 14.570057072199504
RMSE for test set: 15.143103582113675


Let's stick with our parameters of 200 and 50. Next, let's adjust for the max_features parameter (the maximum number of features random forest considers to split a node). The default max_features is 'auto' so we will try 'sqrt' and 'log2' to see if test RMSE improves. 

In [31]:
from sklearn.preprocessing import StandardScaler

def RF_Fit(X_train,y_train,X_test,y_test,num_estimators,min_samples_leafs,max_feat):
    
    #scaling the data
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
     # create regressor object 
    regressor = RandomForestRegressor(n_estimators = num_estimators, min_samples_leaf=min_samples_leafs, max_features = max_feat, random_state = 0) 
  
    # fit the regressor with x and y data 
    regressor.fit(X_train, y_train)

    tr_pred = regressor.predict(X_train) 
    te_pred = regressor.predict(X_test) 
    
    print("RMSE for training set:", sqrt(mean_squared_error(y_train, tr_pred)))
    print("RMSE for test set:", sqrt(mean_squared_error(y_test, te_pred)))

In [32]:
RF_Fit(X_train,y_train,X_test,y_test,200,50,'sqrt')

RMSE for training set: 15.175714465966099
RMSE for test set: 15.141332288735486


In [33]:
RF_Fit(X_train,y_train,X_test,y_test,200,50,'log2')

RMSE for training set: 15.175714465966099
RMSE for test set: 15.141332288735486


The default for max_features is 'auto'. If we recall from above, this achieved a test RMSE of 15.14 and train RMSE of 14.57 All of these models perform equally well. There is a small difference in the error for test and training set - indicating our model has found the sweet spot in the bias variance trade-off. Log2 and sqrt both diminish the variance of our model. Let's just stick with log2


#### Separating weekends and weekdays to see if it improves model performance

In [34]:
weekday = [0,1,2,3,4]
weekday_tr = tr[tr['Weekday'].isin(weekday)]
weekday_te = te[te['Weekday'].isin(weekday)]

In [35]:
#selecting X and Y variables
y_train = weekday_tr['tv delta']
X_train = weekday_tr[['Temperature', 'holiday', 'prev value','twice prev value','day shift','month shift']]

y_test = weekday_te['tv delta']
X_test = weekday_te[['Temperature', 'holiday', 'prev value','twice prev value','day shift','month shift']]

In [36]:
RF_Fit(X_train,y_train,X_test,y_test,200,50,'log2')

RMSE for training set: 16.27732440729296
RMSE for test set: 15.769662053532306


The model performed worse on weekday data alone. Let's leave the weekdays and weekends together. 

### XGBoost 

Can we improve our model's performance?


Random Forest achieved:
- RMSE for training set: 15.175714465966099
- RMSE for test set: 15.141332288735486

In [37]:
import xgboost as xgb

In [38]:
#selecting X and Y variables
y_train = tr['tv delta']
X_train = tr[['Temperature', 'holiday', 'prev value','twice prev value','day shift','month shift']]

y_test = te['tv delta']
X_test = te[['Temperature', 'holiday', 'prev value','twice prev value','day shift','month shift']]

In [39]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)
# model training
params = {'seed': 33,
          'objective': 'reg:squarederror',
          'silent': 0,
          'nthread': 1,
          'max_depth': 2,
          'learning_rate': 0.1}
num_round = 5000
evallist = [(dtrain, 'train'), (dtest, 'eval')]
bst = xgb.train(params, dtrain, num_round, evallist, early_stopping_rounds=50)
tr['pred'] = bst.predict(xgb.DMatrix(X_train))
te['pred'] = bst.predict(xgb.DMatrix(X_test))

[11:54:13] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:130.23007	eval-rmse:110.43436
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[1]	train-rmse:117.69630	eval-rmse:98.91894
[2]	train-rmse:106.45283	eval-rmse:89.29782
[3]	train-rmse:96.31318	eval-rmse:80.06074
[4]	train-rmse:87.21256	eval-rmse:71.87453
[5]	train-rmse:79.04758	eval-rmse:64.67923
[6]	train-rmse:71.70111	eval-rmse:58.65943
[7]	train-rmse:65.12905	eval-rmse:53.10919
[8]	train-rmse:59.23132	eval-rmse:48.04398
[9]	train-rmse:53.96944	eval-rmse:43.51214
[10]	train-rmse:49.25400	eval-rmse:39.7

The random forest model actually performed better than XGBoost. 
XGBoost had the following metrics:
- train-rmse:15.26746	
- test-rmse:15.22363

### The best model: 

In [43]:
y_train = tr['tv delta']
X_train = tr[['Temperature', 'holiday', 'prev value','twice prev value','day shift','month shift']]

y_test = te['tv delta']
X_test = te[['Temperature', 'holiday', 'prev value','twice prev value','day shift','month shift']]

#scaling the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
    
# create regressor object 
regressor = RandomForestRegressor(n_estimators = 200, min_samples_leaf=50, max_features = 'log2', random_state = 0) 
  
# fit the regressor with x and y data 
regressor.fit(X_train, y_train)

tr_pred = regressor.predict(X_train) 
te_pred = regressor.predict(X_test) 
    
print("RMSE for training set:", sqrt(mean_squared_error(y_train, tr_pred)))
print("RMSE for test set:", sqrt(mean_squared_error(y_test, te_pred)))

RMSE for training set: 15.175714465966099
RMSE for test set: 15.141332288735486


In [45]:
feature_list = ['Temperature', 'holiday', 'prev value','twice prev value','day shift','month shift']
# Get numerical feature importances
importances = list(regressor.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: prev value           Importance: 0.9
Variable: Temperature          Importance: 0.03
Variable: day shift            Importance: 0.03
Variable: twice prev value     Importance: 0.02
Variable: month shift          Importance: 0.02
Variable: holiday              Importance: 0.0


In [46]:
tr['pred'] = tr_pred
te['pred'] = te_pred

In [48]:
te.to_csv('test_predictions.csv')
tr.to_csv('train_predictions.csv')

### Testing a neural net

In [49]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor

#scaling data 
from sklearn.preprocessing import StandardScaler


mlp = MLPRegressor(solver = 'adam',max_iter = 5000, batch_size = 64, learning_rate_init = 0.005 )

mlp.fit(X_train,y_train)



#add in dropout 

MLPRegressor(activation='relu', alpha=0.0001, batch_size=64, beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.005, max_fun=15000, max_iter=5000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [50]:
tr_pred = mlp.predict(X_train) 
te_pred = mlp.predict(X_test) 
    
print("RMSE for training set:", sqrt(mean_squared_error(y_train, tr_pred)))
print("RMSE for test set:", sqrt(mean_squared_error(y_test, te_pred)))

RMSE for training set: 15.381277809035472
RMSE for test set: 14.994611090819559


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor

param_grid = [
        {
            'activation' : ['tanh', 'relu'],
            'solver' : ['adam'],
            'hidden_layer_sizes': [(5,),(15,),(50,),(100,)],
            'batch_size': [20,200,600]
        }
       ]

clf = GridSearchCV(MLPRegressor(), param_grid, cv=3,
                           scoring='neg_root_mean_squared_error')
clf.fit(X_train,y_train)




print("Best parameters set found on development set:")
print(clf.best_params_)

C:\Users\india\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\india\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\india\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\india\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv